### 1. Importing Packages and Image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image = cv2.imread("road.jpg")

cv2.imshow("Original", image)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
lane_image = np.copy(image)

### 2. Grayscale Conversion

In [ ]:
gray = cv2.cvtColor(lane_image, cv2.COLOR_RGB2GRAY)

cv2.imshow("Image", gray)

cv2.waitKey()
cv2.destroyAllWindows()

### 3. Gaussian Blur

In [ ]:
blur = cv2.GaussianBlur(gray, (5, 5), 0)

cv2.imshow("Image", blur)

cv2.waitKey()
cv2.destroyAllWindows()

### 4. Canny 

In [ ]:
#canny = cv2.Canny(blur, 30, 90)
canny = cv2.Canny(blur, 50, 150)

cv2.imshow("Image", canny)

cv2.waitKey()
cv2.destroyAllWindows()

### 5. Region of Interest

In [ ]:
def cannyImage(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)    
    blur = cv2.GaussianBlur(image, (5, 5), 0)
    #canny = cv2.Canny(blur, 30, 90)
    canny = cv2.Canny(blur, 50, 150)
    
    return canny

In [ ]:
canny = cannyImage(image)

cv2.imshow("Image", canny)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
plt.imshow(canny)
plt.show()

In [ ]:
def region_of_interest(image):
    
    height = image.shape[0]
    #polygons = np.array([[(250, height), (1500, height), (800, 400)]])
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    
    return mask

In [ ]:
cv2.imshow("Image", region_of_interest(canny))

cv2.waitKey()
cv2.destroyAllWindows()

### 6. Bitwise And

In [ ]:
def region_of_interest(image):
    
    height = image.shape[0]
    #polygons = np.array([[(250, height), (1500, height), (800, 400)]])
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    
    return masked_image

In [ ]:
cropped_image = region_of_interest(canny)
cv2.imshow("Image", cropped_image)

cv2.waitKey()
cv2.destroyAllWindows()

### 7 & 8. Hough Transform

In [ ]:
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap = 5)

In [ ]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
            
    return line_image

In [ ]:
line_image = display_lines(lane_image, lines)

cv2.imshow("Image", line_image)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)

cv2.imshow("Image", combo_image)

cv2.waitKey()
cv2.destroyAllWindows()

### 9. Optimization

In [ ]:
def average_slope_intercept(image, lines):
    
    left_fit = []
    right_fit = []
    
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
        
    left_fit_average = np.average(left_fit, axis = 0)
    right_fit_average = np.average(right_fit, axis = 0)
    left_lane = make_coordinates(image, left_fit_average)
    right_lane = make_coordinates(image, right_fit_average)
    return np.array([left_lane, right_lane])

In [ ]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * (3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

In [ ]:
averaged_lines = average_slope_intercept(lane_image, lines)

In [ ]:
line_image = display_lines(lane_image, averaged_lines)

cv2.imshow("Image", line_image)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)

cv2.imshow("Image", combo_image)

cv2.waitKey()
cv2.destroyAllWindows()

### 10. Finding Lane Lines

In [ ]:
cap = cv2.VideoCapture("test2.mp4")

while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = cannyImage(frame)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap = 5)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow("Image", combo_image)
    cv2.waitKey(1)
    
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def cannyImage(image):    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)    
    blur = cv2.GaussianBlur(image, (5, 5), 0)
    canny = cv2.Canny(blur, 30, 90)    
    return canny


def region_of_interest(image):    
    height = image.shape[0]
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)    
    return masked_image


def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10) 
    else: 
        pass
    return line_image


def average_slope_intercept(image, lines):
    
    left_fit = []
    right_fit = []
    
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
        
    left_fit_average = np.average(left_fit, axis = 0)
    right_fit_average = np.average(right_fit, axis = 0)
    left_lane = make_coordinates(image, left_fit_average)
    right_lane = make_coordinates(image, right_fit_average)
    return np.array([left_lane, right_lane])


def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * (3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])


cap = cv2.VideoCapture("test2.mp4")

while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = cannyImage(frame)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 60, np.array([]), minLineLength = 10, maxLineGap = 5)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow("Image", combo_image)
    cv2.waitKey(1)
    
cv2.destroyAllWindows()

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
